<a href="https://colab.research.google.com/github/guhdalla/IdentificarPassaporte/blob/main/IdentificarPassaporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok

     |████████████████████████████████| 745 kB 5.1 MB/s 


In [2]:
!pip install -q streamlit

     |████████████████████████████████| 10.1 MB 4.9 MB/s 
     |████████████████████████████████| 76 kB 5.7 MB/s 
     |████████████████████████████████| 164 kB 53.7 MB/s 
     |████████████████████████████████| 181 kB 50.6 MB/s 
     |████████████████████████████████| 111 kB 53.7 MB/s 
     |████████████████████████████████| 4.3 MB 36.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 131 kB 57.0 MB/s 
     |████████████████████████████████| 130 kB 68.4 MB/s 
     |████████████████████████████████| 793 kB 28.6 MB/s 
     |████████████████████████████████| 428 kB 47.8 MB/s 
     |████████████████████████████████| 381 kB 48.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

In [3]:
!pip install -q streamlit_ace

     |████████████████████████████████| 3.6 MB 5.1 MB/s 


In [4]:
!streamlit run app.py &>/dev/null&

In [5]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 50 kB 6.3 MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149861 sha256=2e9ac54328541c9a5a34629ca95c5c1e9dafc83a6f81c9f5cb6e3f6bf385387d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-levenshtein


In [6]:
from pyngrok import ngrok

In [19]:
%%writefile app.py

from fuzzywuzzy import process
import streamlit as st
import re
import json
import requests

def main():

  def getPaises():
    response = requests.get('https://guhdalla.github.io/paises.json')
    return response.json()['PAISES']
      
  def getUrlByPais(paisField):
    paises = getPaises();
    for pais in paises:
      if pais["PAIS"].upper() == paisField.upper():
        return pais["URL"]

  def containPassaporteInUrl(url, sigla, passaporte):
    response = requests.get(url)
    json = response.json()[sigla.lower()]
    print(sigla.lower())
    for i in json:
      if int(i[sigla.lower()]) == int(passaporte):
        return True
    return False

  def containNameInInterpol(nome):
    response = requests.get("https://cspinheiro.github.io/interpol.json")
    json = response.json()["interpol"]
    nomes = []
    for i in json:
      nomes.append(i["interpol"])

    result = process.extract(nome, nomes)
    print(result)
    for text in result: 
        if text[1] > 86:
          return True
        else:
          return False
    

  def service(passaporte, nome):
    pais = re.sub("[0-9]{9}", "",passaporte)
    passaporte = re.sub("[a-zA-Z]{3}", "",passaporte)

    url = getUrlByPais(pais);
    if url is None:
      return st.error("Pais inexistente!")
    isPassaporte = containPassaporteInUrl(url, pais, passaporte)
    isWanted = containNameInInterpol(nome)
    print("Pass", isPassaporte)
    print("Wanted", isWanted)
    if not isPassaporte and not isWanted:
      st.success("Permitido.")
    else:
      if isWanted:
        st.error("Pessoa sujeito a investigação!")
      else:
        st.error("Nao permitido!")


  def verifyNome(nome):
    if nome is "":
      return False
    match = re.search("[0-9]", nome)
    if not match:
      return True
    else:
      return False

  def verifyPassaporte(passaporte):
    match = re.search("([a-zA-Z]{3})([0-9]{9})", passaporte)
    if match:
      return True
    else:
      return False

  with st.form("form_migracao"):
    st.write("MIGRACAO")
    passaporte = st.text_input('Passaporte')
    nome = st.text_input('Nome')

    submitted = st.form_submit_button("VERIFICAR")
    if submitted:
      if verifyPassaporte(passaporte) and verifyNome(nome):
        service(passaporte, nome)
      else:
        st.write("Passaporte ou nome foram digitados incorretamente! Tente Novamente.")

if __name__=='__main__': 
    main()  

Overwriting app.py


In [20]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-04 22:47:14.059 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.416s
your url is: https://soft-aliens-flash-35-185-63-121.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.185.63.121:8501

fra
[('Luzia dos Anjos ', 56), ('Priscila dos Santos Pereira', 54), ('Isabelle dos Anjos Pinheiro', 54), ('Fabiana Oliveira Ribeiro', 51), ('Luzivan de Oliveira ', 45)]
Pass True
Wanted False
ind
[('Fernando de Villas Boas ', 100), ('Bruna de Villas Boas ', 84), ('Pedro de Villas Boas ', 84), ('Marianna de Villas Boas ', 78), ('Maurício de Villas Boas ', 76)]
Pass False
Wanted True
irq
[('Fábio Oliveira ', 86), ('Larissa Oliveira ', 67), ('Isabelle de Oliveira ', 65), ('Camila de Oliveira ', 65), ('Pedro de Oliveira ', 65)]
Pass False
Wanted False
fra
[('Lucas Silva Leão', 100), ('Luana Silva Leão', 87), ('Pietra da Silva Rebouças', 86), ('Patrícia da Silva Ribeiro', 86), ('Maurício d